# Imports

In [70]:
import pandas as pd; import seaborn as sb; import statsmodels.formula.api as smf; import pingouin;
from statannot import add_stat_annotation; from itertools import combinations; import os
from analysis_helpers import *



import warnings
warnings.filterwarnings('ignore')

# Load Data

In [71]:
merged = pd.DataFrame.from_csv('gaze_coords_with_ratings.csv')
eye_behav_full = pd.DataFrame.from_csv('eye_behav_full.csv')
eye_behav = pd.DataFrame.from_csv('eye_behav.csv')
total = pd.DataFrame.from_csv('data_with_correct_scaled_gaze.csv')

In [72]:
# eye_behav_full[eye_behav_full[eye_behav_full['Trial Type']=='Presentation']['Subject']==8.0]['Trial']

In [73]:
def eye_read(dir, out_str):
    '''
    input:  directory with gaze data
    output: gaze df, also saves pickle
    '''
    # "/Users/kirstenziman/Documents/attention-memory-task/data/"
    all_gaze = []

    for b in os.listdir(dir):
        print(b)
        dr = dir+b
        initial = eye_intial( dr + "/eye_data/")
        pres = pres_gaze_image(dr, initial, b[0:2])
        subject_gaze = pd.concat([pd.concat(pres[x]) for x in pres.keys()])
        all_gaze.append(subject_gaze)

    df = pd.concat(all_gaze)
    eye_df = df

    # convert to centimeters
    df['av_x_coord'] = (59.8/2048)*(df[['xRaw_righteye', 'xRaw_lefteye']].mean(axis=1))
    df['av_y_coord'] = (33.6/1152)*(df[['yRaw_righteye', 'yRaw_lefteye']].mean(axis=1))
    # "/Users/kirstenziman/Documents/attention-memory-task/gaze_May_update.pkl"
    pickle.dump(df, open(out_str, "wb" ))

    return(df)

class parseFile():
    def __init__(self, file):
        self.file = file
    def parse(self):
        data = open(self.file).read()
        return(data)

def load(path):
    '''
    input: path to directory containing eye track data
    output: raw parsed eye data
    '''

    data = []
    files = [f for f in os.listdir(path)]

    for x in files:
        #if os.path.isfile(path+x):
        newFile = parseFile(path+x)
        data1 = newFile.parse()

        for a,b in zip(['true','false'], ['True', 'False']):
            data1 = data1.replace(a, b)

        data1 = data1.split('\n')
        data1 = [x for x in data1 if "tracker" in x]
        data.extend(data1)

    return(data)

In [74]:
# def eye_intial(path):
#     ''' reads in raw eye gaze data
#         outputs eye gaze dataframe, timestamps in GMT
#     '''

#     data = []

#     for x in os.listdir(path):
#         newFile = parseFile(path+x)
#         data1 = newFile.parse()

#         for a,b in zip(['true','false'], ['True', 'False']):
#             data1 = data1.replace(a, b)

#         data1 = data1.split('\n')
#         data1 = [x for x in data1 if "tracker" in x]
#         data.extend(data1)

#     dict_list = [as.literal_eval(x) for x in data]
#     dict_list = [x['values']['frame'] for x in dict_list if 'values' in x and 'frame' in x['values']]
#     df = pd.DataFrame(dict_list)

#     for eye in ['righteye','lefteye']:
#         for coord in ['x','y']:
#             df[coord+'Raw_'+eye] = [df[eye][row]['raw'][coord] for row in df.index.values]

#     df['av_x_coord'] = df[['xRaw_righteye', 'xRaw_lefteye']].mean(axis=1)
#     df['av_y_coord'] = df[['yRaw_righteye', 'yRaw_lefteye']].mean(axis=1)
#     df['timestamp']=[time.mktime(time.strptime(x[:], "%Y-%m-%d %H:%M:%S.%f")) for x in df['timestamp']]

#     return(df)

In [75]:
#eye_data = eye_read('/Users/kirstenziman/Documents/github/attention-memory-task/data/','eye_data_exp1_actual.pkl')

In [76]:
min(eye_behav['av_x_coord'])

8.675401782226562

In [77]:
with open('eye_data_exp1.pkl','rb') as fp:
    eye_data_exp1 = pickle.load(fp)
    
eye_data_exp1 = eye_data_exp1[eye_data_exp1['av_x_coord']<59.8]
eye_data_exp1['Subject'] = eye_data_exp1['Subject'].map(lambda x: x.replace('_',''))

In [78]:
max(eye_data_exp1.groupby(['Subject','Run','Trial'], as_index=False).mean()['av_x_coord'])

39.622601329706754

In [79]:
tr = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]*8*30
df_list['Trial']=np.nan
df_list.loc[df_list['Trial Type']=='Presentation','Trial']=tr
df_list.head()

NameError: name 'df_list' is not defined

In [ ]:
grouped_eye = eye_data_exp1.groupby(['Subject','Run','Trial'], as_index=False).mean()

grouped_eye['Subject'] = pd.to_numeric(grouped_eye['Subject'])
grouped_eye['Run']     = pd.to_numeric(grouped_eye['Run'])
grouped_eye['Trial']   = pd.to_numeric(grouped_eye['Trial'])


merged_pres = grouped_eye.merge(df_list, on=['Subject', 'Run', 'Trial'])

In [ ]:
def add_gaze(df):
    '''
    input: df containing pres and mem from single run
    output: df with string in 'Attention Level' column in each Memory trial row
    '''

    for index,row in df.iterrows():
        if row['Trial Type']=='Memory':
            mem_image = row['Memory Image']
            for cue in ['Cued ', 'Uncued ']:
                for cat in ['Face', 'Place']:
                    if df.loc[df[cue+cat] == mem_image].shape[0]!=0:
                        df['av_x_coord'][index]=df.loc[df[cue+cat] == mem_image]['av_x_coord']  
                        df['Cued Side'][index] = df.loc[df[cue+cat] == mem_image]['Cued Side'].item()  

    mem_mask = df['Trial Type']=='Memory'
    df.loc[mem_mask,'av_x_coord'] = df.loc[mem_mask,'av_x_coord'].fillna(np.nan)

    return(df)

In [80]:
combine = pd.concat([merged_pres, df_list[df_list['Trial Type']=='Memory']])

NameError: name 'merged_pres' is not defined

In [81]:
subs = []

for sub in combine['Subject'].unique():
    print(sub)
    name_str = 'subject_' + str(sub)
    subs.append(add_gaze(combine[combine['Subject']==sub]))
    pickle.dump(sub, open(name_str, "wb" ))
    
pickle.dump(subs, open("subs_real1.p", "wb" ))

NameError: name 'combine' is not defined

# Add Gaze Function

In [82]:
#def add_gaze(df):
def add_side(df):
    '''
    input: df containing pres and mem from single run
    output: df with string in 'Attention Level' column in each Memory trial row
    '''

    for index,row in df.iterrows():
        if row['Trial Type']=='Memory':
            mem_image = row['Memory Image']
            for cue in ['Cued ', 'Uncued ']:
                for cat in ['Face', 'Place']:
                    if df.loc[df[cue+cat] == mem_image].shape[0]!=0:
                        df['av_x_coord'][index]=df.loc[df[cue+cat] == mem_image]['av_x_coord']  
                        df['Cued Side'][index] = df.loc[df[cue+cat] == mem_image]['Cued Side'].item()  

    mem_mask = df['Trial Type']=='Memory'
    df.loc[mem_mask,'av_x_coord'] = df.loc[mem_mask,'av_x_coord'].fillna(np.nan)

    return(df)

In [83]:
# total=[]

# for s in full['Subject'].unique():
#     print(s)
    
#     for r in full['Run'].unique():
#         #print(r)
        
#         total.append(add_side(full[(full['Subject']==s) & (full['Run']==r)]))

In [84]:
# total = pd.concat(total)

In [85]:
#total['Cued Side']

In [86]:
# total['Subject'].unique()
# df['Cued Side'][index] = df.loc[df[cue+cat] == mem_image]['Cued Side']

In [87]:
# everything on the right is okay

# everything on the left nees a switch

total.loc[total['Cued Side']=='<','scaled_x_coord'] = abs(total[total['Cued Side']=='<']['scaled_x_coord'])
total.loc[total['Attention Level']=='Category','scaled_x_coord'] = -total[total['Attention Level']=='Category']['scaled_x_coord']
total.loc[total['Attention Level']=='None','scaled_x_coord'] = -total[total['Attention Level']=='None']['scaled_x_coord']

In [88]:
total = pd.DataFrame.from_csv('data_with_correct_scaled_gaze.csv')

In [89]:
total_g = total.groupby(['Subject','Attention Level', 'Familiarity Rating'],as_index=False).mean()
total_g.head()

,Subject,Attention Level,Familiarity Rating,Attention Button,Attention Reaction Time (s),Cue Validity,Familiarity Reaction Time (s),Post Invalid Cue,Pre Invalid Cue,Rating History,...,fix,state,time,timestamp,xRaw_lefteye,xRaw_righteye,yRaw_lefteye,yRaw_righteye,FamiliarityRating,scaled_x_coord
0,0.0,Category,1.0,NaN,NaN,NaN,1.103636,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,-0.009935
1,0.0,Category,2.0,NaN,NaN,NaN,1.613727,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,-0.013504
2,0.0,Category,3.0,NaN,NaN,NaN,1.487000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,-0.021296
3,0.0,Category,4.0,NaN,NaN,NaN,1.250833,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,-0.003771
4,0.0,Full,1.0,NaN,NaN,NaN,1.227889,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.006154


In [90]:
sb.set_style('white')


sb.set_style('white')


f = sb.violinplot(x='Familiarity Rating', y='scaled_x_coord', 
              data=total_g[total_g['Attention Level']!='Novel'], hue='Attention Level')

plt.grid(False)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.xlabel('Familiarity Rating',    fontsize = 20)
plt.ylabel('Horizontal Gaze Drift', fontsize = 20)
f.set(ylim=(-1, 1))
plt.savefig('av_x_gaze_exp2_mixy.pdf')




In [91]:
total_g['Subject'].unique()

array([  0.,   1.,   2.,   3.,   4.,   5.,   7.,   8.,  10.,  12.,  13.,
        14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,
        25.,  26.,  27.,  28.,  29.,  33.,  34., 327.])

In [92]:
anova_dat = total_g.groupby(['Attention Level','Familiarity Rating','Subject'],as_index=False).mean()

In [104]:
pingouin.rm_anova(dv='Familiarity Rating', within=['Attention Level', 'scaled_x_coord'], subject='Subject', 
                  data=anova_dat[anova_dat['Attention Level']!='Novel'])

,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,np2,eps
0,Attention Level,88.042,3,81,29.347,1.811,0.15162,0.165856,0.063,0.779
1,scaled_x_coord,57938.502,421,11367,137.621,-27.768,1.00000,NaN,36.143,NaN
2,Attention Level * scaled_x_coord,-43541.919,1263,34101,-34.475,-28.964,1.00000,NaN,14.749,NaN


# Regression

In [105]:
total.loc[total['Attention Level']=='Novel','scaled_x_coord']=0
#total[total['Attention Level']=='Novel']['scaled_x_coord']

In [106]:
total['FamiliarityRating'] = total['Familiarity Rating']
d = []

for subject in total['Subject'].unique()[:-1]: 
    print(subject)
    
    sub  = []
    data = total[(total['Subject']==subject) & (total['Trial Type']=='Memory') ]
    reg  = smf.ols(formula = "FamiliarityRating ~ scaled_x_coord", data = data).fit()
    data = data.reset_index()
    d.append(pd.concat([data, pd.DataFrame({'gaze_resid':reg.resid})], axis=1, sort=False))
    
gaze_reg = pd.concat(d)

8.0
15.0
1.0
25.0
12.0
4.0
28.0
19.0
21.0
33.0
7.0
26.0
14.0
2.0
16.0
27.0
23.0
18.0
29.0
13.0
5.0
24.0
0.0
34.0
22.0
17.0
3.0
10.0
20.0


In [107]:
# total['FamiliarityRating'] = total['Familiarity Rating']
# d=[]

# for subject in total['Subject'].unique(): 
#     sub=[]
#     data = total[total['Subject']==subject]
#     reg = smf.ols(formula = "FamiliarityRating ~ scaled_x_coord", data = data).fit()
#     data = data.reset_index()
#     d.append(pd.concat([data, pd.DataFrame({'gaze_resid':reg.resid})], axis=1, sort=False))
    
# gaze_reg = pd.concat(d)

In [108]:
missing_sub = add_side(eye_behav_full[eye_behav_full['Subject']==327])

# Load Data

In [109]:
full = pd.DataFrame.from_csv('av_x_coord_in_mem_trials.csv')

In [110]:
total = pd.concat([total[total['Subject']!=327], missing_sub])

In [111]:
# full['scaled_x_coord'] = (full['av_x_coord'] - (59.8/2))/(59.8/2)

In [112]:
for sub in full['Subject'].unique():
    
    for r in full['Run'].unique():
        
        dat = full[(full['Subject']==s) & full[(full['Run']==r)]]
        mem_dat  = dat[dat['Trial Type']=='Memory']
        pres_dat = dat[dat['Trial Type']=='Presentation']
        
        for row in mem_dat.iterrows():
            
             
             
             
             
    # for every memory image

    # look at cued side during presentation
    
    
    
    # if cued side == '<' 
    #       if av_x_coord < 0
    #           if attn level Full or SIde --> abs val (x coord)
    #.      if av_x_coord > 0
    #.          if attn level Cat or None --> 

SyntaxError: unexpected EOF while parsing (<ipython-input-112-84ff75edd88d>, line 25)

In [ ]:
# full['scaled_x_coord']

# Gaze ANOVA

In [ ]:
attn_group = full.groupby(['Subject','Attention Level'], as_index=False).mean()

In [ ]:
# ORGANIZE DATA
pingouin.rm_anova(dv='Familiarity Rating', within=['Attention Level', 'scaled_x_coord'], subject='Subject', 
                  data=attn_group[attn_group['Attention Level']!='Novel'])

In [ ]:
attn_group.head()

In [ ]:
pingouin.rm_anova(dv='Adjusted', within=['Attention Level'], subject='Subject', data=reg_attn_group)

## Gaze Regression - issue: what is wrong with last subject ??

### obtain residuals

In [ ]:
# full[full['Subject']==327]

In [ ]:
full['Subject'].unique()#[0:-1]

full=total

In [ ]:
full['FamiliarityRating'] = full['Familiarity Rating']
d=[]

for subject in full['Subject'].unique()[:-1]: 
    sub=[]
    data = full[full['Subject']==subject]
    reg = smf.ols(formula = "FamiliarityRating ~ scaled_x_coord", data = data).fit()
    data = data.reset_index()
    d.append(pd.concat([data, pd.DataFrame({'gaze_resid':reg.resid})], axis=1, sort=False))
    
gaze_reg = pd.concat(d)

### subtract residuals, group data

In [ ]:
# gaze_reg['gaze_resid']

In [ ]:
gaze_reg['Adjusted'] = gaze_reg['Familiarity Rating']-gaze_reg['gaze_resid']

reg_attn_group = gaze_reg.groupby(['Subject','Attention Level'],as_index=False).mean()
reg_cat_group  = gaze_reg.groupby(['Subject','Attention Level','Category'],as_index=False).mean()
face  = reg_cat_group[reg_cat_group['Category']=='Face']
place = reg_cat_group[reg_cat_group['Category']=='Place']

## Plot Adjust Scores (Full)

### plot params

In [ ]:
x='Attention Level'; y='Adjusted'; order=['Full','Side','Category','None','Novel']

In [ ]:
# total.groupby(['Subject'],as_index=False).mean()['scaled_x_coord']

In [ ]:
l

In [ ]:
from itertools import combinations

ax = sb.violinplot(x=x, y=y, data=reg_attn_group, order=order)

pair = combinations(reg_attn_group['Attention Level'].unique(),2) 
l = [i for i in pair]


# ax = sb.barplot(x=x, y=y, data=data, order=['Full','Side','Category','None','Novel'])        

add_stat_annotation(ax, data=reg_attn_group, x=x, y=y, order=['Full','Side','Category','None','Novel'], 
                    boxPairList=l, test='t-test_paired', textFormat='star', loc='outside', verbose=2)

In [ ]:
reg_attn_group.groupby(['Attention Level']).mean()['Adjusted']

In [ ]:
pingouin.rm_anova(dv='Adjusted', within=['Attention Level'], subject='Subject', 
                  data=reg_attn_group[reg_attn_group['Attention Level']!='Novel'])

## Plot Adjusted Scores (Face)

In [ ]:
# from itertools import combinations
# from statannot import add_stat_annotation

# pair = combinations(groupy['Attention Level'].unique(),2) 
# l = [i for i in pair]

# ax = sb.barplot(x=x, y=y, data=data, order=['Full','Side','Category','None','Novel'])        

# add_stat_annotation(ax, data=face, x=x, y=y, order=['Full','Side','Category','None'], 
#                    boxPairList=l, test='t-test_paired', textFormat='star', loc='outside', verbose=2)

In [ ]:
sb.barplot(x=x, y=y, data=face, order=order)
plt.show()


## Plot Adjusted Scores (Place)

In [ ]:
sb.barplot(x=x, y=y, data=place, order=order)

In [ ]:
import scipy 

pair = combinations(groupy_face['Attention Level'].unique(),2) 
   
for i in list(pair): 
    print(i)
    p = scipy.stats.ttest_rel(groupy_face[groupy_face['Attention Level']==i[0]]['Adjusted'], 
                          groupy_face[groupy_face['Attention Level']==i[1]]['Adjusted'])
    print(p)
    print(' ')

In [ ]:
sb.barplot(x='Attention Level',y='Familiarity Rating', 
              data=groupy_face, order=['Full','Side','Category','None','Novel'])

In [ ]:
sb.barplot(x='Attention Level',y='Adjusted', 
              data=groupy_face, order=['Full','Side','Category','None','Novel'])

In [ ]:
sb.barplot(x='Attention Level',y='Familiarity Rating', order=['Full','Side','Category','Novel','None'],
              data=groupy_place)

In [ ]:
sb.barplot(x='Attention Level',y='Adjusted', 
              data=groupy_place, order=['Full','Side','Category','None','Novel'])

In [ ]:
sb.barplot(x='Attention Level',y='Adjusted', 
              data=groupy_face)

In [ ]:
pingouin.rm_anova(dv='Adjusted', within=['Attention Level'], subject='Subject', data=gaze_reg.groupby(['Subject','Attention Level'],as_index=False).mean())

In [ ]:
import scipy.stats as stats

data=gaze_reg.groupby(['Subject','Attention Level'],as_index=False).mean()

stats.ttest_rel(data[data['Attention Level']=='Category']['Adjusted'], 
                data[data['Attention Level']=='None']['Adjusted'])